In [ ]:
%config InlineBackend.figure_format='retina'
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))



### General imports
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.signal import medfilt
from scipy.signal import resample
from scipy.interpolate import interp1d
import glob
from importlib import reload
import healpy as hp
import sys

plt.rc('figure',figsize=(16,10))
plt.rc('font',size=12)

dirtemplibs = [os.environ['QUBIC_DATADIR']+'scripts/DiversJC/CalibSalta/']
for rep in dirtemplibs:     
    if rep not in sys.path:
        sys.path.append(rep)

#### Local files that will need to be installed in the Qubic Libs
import fitting as fit


In [ ]:
def generate_gaussian_realization_2D(power_spectrum, grid_size, seed=None, grid_scale=1):
    """
    Generate a 2D Gaussian realization from a given power spectrum (Chat-GPT writtten).

    Parameters:
        power_spectrum (callable): A function P(kx, ky) that defines the power spectrum as a function of kx and ky.
        grid_size (int): The size of the 2D grid (grid_size x grid_size).
        seed (int, optional): Seed for random number generation.

    Returns:
        np.ndarray: A 2D Gaussian realization.
    """
    if seed is not None:
        np.random.seed(seed)

    # Create the kx, ky grids
    kx = np.fft.fftfreq(grid_size) * grid_size * grid_scale
    ky = np.fft.fftfreq(grid_size) * grid_size * grid_scale
    kx, ky = np.meshgrid(kx, ky)

    # Compute the amplitude of the Fourier modes using the power spectrum
    amplitude = np.sqrt(power_spectrum(kx, ky))

    # Generate random Fourier coefficients
    real_part = np.random.normal(0, 1, (grid_size, grid_size))
    imag_part = np.random.normal(0, 1, (grid_size, grid_size))
    fourier_modes = amplitude * (real_part + 1j * imag_part)

    # Enforce Hermitian symmetry to ensure the result is real
    #fourier_modes = (fourier_modes + np.conj(np.flip(np.flip(fourier_modes, axis=0), axis=1))) / 2

    # Perform the inverse Fourier transform to get the 2D realization
    gaussian_realization = np.fft.ifft2(fourier_modes).real

    return gaussian_realization


# Define an example power spectrum
def example_power_spectrum(kx, ky):
    k2 = kx**2 + ky**2
    return np.exp(-k2 / 10.0)  # Gaussian power spectrum for demonstration

def kolmogorov_power_spectrum(kx, ky, k0=1, power=-8/3):
    """
    Kolmogorov power spectrum with a power law.

    Parameters:
        kx (np.ndarray): The wave numbers in the x-direction.
        ky (np.ndarray): The wave numbers in the y-direction.
        k0 (float, optional): The smallest wave number (injection scale) to avoid singularity at k=0. Default is 1.

    Returns:
        np.ndarray: The power spectrum values for the given kx and ky.
    """
    # Compute the wave number magnitude
    k2 = kx**2 + ky**2
    k = np.sqrt(k2)

    # Avoid division by zero at k=0 by setting a minimum cutoff at k0
    k = np.maximum(k, k0)

    # Kolmogorov power spectrum proportional to k^power
    return k**power


def pointing(nn, dx, dy, x0, y0, nscans=10):
    ptgx = x0 + dx * np.sin(nscans * np.linspace(0,1,nn)*2*np.pi)
    ptgy = y0 + np.linspace(0, dy, nn)
    return ptgx, ptgy

In [ ]:


grid_size = 512
seed = None

x0, y0 = grid_size//2, grid_size//2

realization = generate_gaussian_realization_2D(kolmogorov_power_spectrum, grid_size, seed, grid_scale=1)

ptgx, ptgy = pointing(grid_size, 100, 100, x0, y0, nscans=5)

wind_rms = 20
windx = realization[np.random.randint(grid_size),:]
windx = windx * (wind_rms / np.std(windx))
windy = realization[np.random.randint(grid_size),:]
windy = windy * (wind_rms / np.std(windy))

# Plot the realization
plt.figure()
plt.imshow(realization, origin='lower', extent=(0, grid_size, 0, grid_size), cmap='viridis')
plt.plot(windx + x0, windy + y0, 'r', label='Wind motion')
plt.plot(ptgx, ptgy, 'k', label='Pointing')
plt.colorbar(label='Amplitude')
plt.title('2D Gaussian Realization')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()



In [ ]:
#### Method 1
### Actual pixel pointed at

# def get_signal_direct(ptgx, ptgy, windx, windy, mymap):
#     mypix_x = (np.round(windx+ptgx)).astype(int)
#     mypix_y = (np.round(windy+ptgy)).astype(int)
#     return mymap[mypix_x, mypix_y]

def get_signal_direct(ptgx, ptgy, windx, windy, mymap):
    sh = np.shape(mymap)
    mypix_x = (np.round(windx+ptgx)).astype(int) % sh[0]
    mypix_y = (np.round(windy+ptgy)).astype(int) % sh[1]
    data = np.zeros(len(windx))
    for i in range(len(windx)):
        data[i] = mymap[mypix_x[i], mypix_y[i]]
    return data



mypix_x = np.round(windx+ptgx).astype(int)
mypix_y = np.round(windy+ptgy).astype(int)
plt.figure()
plt.imshow(realization, origin='lower', extent=(0, grid_size, 0, grid_size), cmap='viridis')
plt.plot(mypix_x, mypix_y, 'r', label='pointed pixel')


%timeit toto = get_signal_direct(ptgx, ptgy, windx, windy, realization)
signal = get_signal_direct(ptgx, ptgy, windx, windy, realization)
plt.figure()
plt.plot(signal, label='Direct')
plt.legend()

In [ ]:
#### Method 2 point as usual but shifting the atmospheric map

def get_signal_shiftmap(ptgx, ptgy, windx, windy, mymap):
    mypix_x = np.round(ptgx).astype(int)
    mypix_y = np.round(ptgy).astype(int)
    data = np.zeros(len(windx))
    for i in range(len(windx)):
        dx = -np.round(windx[i]).astype(int)
        dy = -np.round(windy[i]).astype(int)
        data[i] = np.roll(mymap, (dx, dy), axis=(0,1))[mypix_x[i], mypix_y[i]]
    return data


plt.figure()
plt.imshow(realization, origin='lower', extent=(0, grid_size, 0, grid_size), cmap='viridis')
plt.plot(ptgx, ptgy, 'k', label='pointed pixel')

%timeit toto = get_signal_shiftmap(ptgx, ptgy, windx, windy, realization)
signal_shift = get_signal_shiftmap(ptgx, ptgy, windx, windy, realization)
plt.figure()
plt.plot(signal, label='Direct')
plt.plot(signal_shift,'--', label='Roll')
plt.legend()

In [ ]:
### Method 3: roll by hand by calculating relevant indices on the fly
def get_signal_shiftmap_indices(ptgx, ptgy, windx, windy, mymap):
    sh = np.shape(mymap)
    mypix_x = np.round(ptgx).astype(int)
    mypix_y = np.round(ptgy).astype(int)
    dx = np.round(windx).astype(int)
    dy = np.round(windy).astype(int)
    data = np.zeros(len(windx))
    for i in range(len(windx)):
        data[i] = mymap[(mypix_x[i]+dx[i]) % sh[0], (mypix_y[i]+dy[i]) % sh[1] ]
    return data


plt.figure()
plt.imshow(realization, origin='lower', extent=(0, grid_size, 0, grid_size), cmap='viridis')
plt.plot(ptgx, ptgy, 'k', label='pointed pixel')

%timeit toto = get_signal_shiftmap_indices(ptgx, ptgy, windx, windy, realization)
signal_shift_indices = get_signal_shiftmap_indices(ptgx, ptgy, windx, windy, realization)
plt.figure()
plt.plot(signal, label='Direct')
plt.plot(signal_shift,'--', label='Roll')
plt.plot(signal_shift_indices,':', label='Roll Indices')
plt.legend()

## Bruit en 1/f

In [ ]:
grid_size = 512
seed = None

x0, y0 = grid_size//2, grid_size//2

realization = generate_gaussian_realization_2D(kolmogorov_power_spectrum, grid_size, seed, grid_scale=1)

nptg = 2**13
dx = 200
dy = 200
ptgx, ptgy = pointing(nptg, dx, dy, x0, y0, nscans=5)

winds = [0, 10, 100, 1000]

for i in np.arange(len(winds)):
    wind_rms = winds[i]
    windx = resample(realization[np.random.randint(grid_size),:], nptg)
    windx = windx * (wind_rms / np.std(windx))
    windy = resample(realization[np.random.randint(grid_size),:], nptg)
    windy = windy * (wind_rms / np.std(windy))
    
    signal = get_signal_direct(ptgx, ptgy, windx, windy, realization)
    plt.subplot(2,1,1)
    plt.plot(signal, label='Direct')
    plt.legend()
    
    ps = np.abs(np.fft.fft(signal))**2
    ff = np.fft.fftfreq(len(signal))
    
    pos = ff>0
    
    plt.subplot(2,1,2)
    plt.plot(ff[pos], ps[pos], '.', alpha=0.5, label='WInd = {:5.2g}'.format(wind_rms))
    plt.xscale('log')
    plt.yscale('log')
    plt.legend()